# Responses API Tutorial: From Introduction to Summary

Below is a comprehensive tutorial and set of examples demonstrating:

1. **How the Responses API works and how it differs from Chat Completions** (particularly around stateful vs. stateless usage).
2. **Examples** of multi-turn conversation (using `previous_response_id` for stateful flows) and built-in tools like web search and file search.
3. **How to disable storage** (`store: false`) if you **do not** want your conversation state to persist on OpenAI’s servers—effectively making it stateless.

---
## 1. Chat Completions (Stateless) vs. Responses (Stateful)

- **Chat Completions**:
  - Typically stateless: each new request must supply the entire conversation history in `messages`.
  - Stored by default only for new accounts; can be disabled.

- **Responses**:
  - By default, **stateful**: each response has its own `id`. You can pass `previous_response_id` in subsequent calls, and the system automatically includes the prior context.
  - Provides built-in **tools** (web search, file search, etc.) that the model can call if relevant.
  - **Stored** by default. If you want ephemeral usage, set `store: false`.

When you get a response back from the Responses API, the returned object differs slightly from Chat Completions:

- Instead of a simple list of message choices, you receive a typed `response` object with top-level fields (e.g. `id`, `output`, `usage`, etc.).
- To continue a conversation, pass `previous_response_id` to the next request.
- If you do **not** want it stored, set `store: false`.


---
## 2. Multi-Turn Flow (Stateful) Example

Using `previous_response_id` means the Responses API will store and automatically incorporate the entire conversation. Here’s a simple demonstration:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from openai import OpenAI

client = OpenAI()


resp1 = client.responses.create(
    model="gpt-4o-mini",
    input="Hello there! You're a helpful math tutor. Could you help me with a question? What's 2 + 2?"
)
print("First response:\n", resp1.output_text)


In [ ]:
resp2 = client.responses.create(
    model="gpt-4o",
    input="Sure. How you you come to this conclusion?",
    previous_response_id=resp1.id
)
print("\nSecond response:\n", resp2.output_text)

---
## 3. Using Built-In Tools

### 3.1 Web Search
Allows the model to gather recent info from the internet if relevant.

In [ ]:
# Example usage of the built-in web_search tool
r1 = client.responses.create(
    model="gpt-4o",
    input="Please find recent positive headlines about quantum computing.",
    tools=[{"type": "web_search"}]  # enabling built-in web search
)
print(r1.output_text)

In [ ]:
# Continue the conversation referencing previous response
r2 = client.responses.create(
    model="gpt-4o",
    input="Interesting! Summarize the second article.",
    previous_response_id=r1.id
)
print("\nFollow-up:\n", r2.output_text)

### 3.2 File Upload + File Search

Below is the corrected snippet showing how to:
1. **Upload** a local PDF (e.g., `dragon_book.pdf`).
2. **Create** a vector store from that file.
3. **Use** `file_search` in the Responses API to reference it.


In [ ]:
upload_resp = client.files.create(
    file=open("dragon_book.txt", "rb"),
    purpose="user_data"
)
file_id = upload_resp.id
print("Uploaded file ID:", file_id)

In [ ]:
client.files.list()

In [ ]:
vstore_resp = client.vector_stores.create(
    name="DragonData",
    file_ids=[file_id]
)
vstore_id = vstore_resp.id
print("Vector store ID:", vstore_id)

In [ ]:
resp1 = client.responses.create(
    model="gpt-4o",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vstore_id],
        "max_num_results": 3
    }],
    input="What Information do you have about red dragons?"
)
print(resp1.output_text)

---
## 4. Disable Storage (Stateless Mode)

Although the Responses API is **stateful** by default, you can make calls **not** store any conversation by setting `store=False`. Then `previous_response_id` won’t work, because no data is retained on OpenAI’s servers.

In [ ]:
# An ephemeral request that won't be stored
ephemeral_resp = client.responses.create(
    model="gpt-4o",
    input="Hello, let's do a single-turn question about geometry.",
    store=False  # ephemeral usage
)
print(ephemeral_resp.output_text)

### LangChain Integration

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", use_responses_api=True)


tool = {"type": "web_search_preview"}


llm_with_tools = llm.bind_tools([tool])


response = llm_with_tools.invoke(input="What was a positive news story from today?")

print("Text content:", response.content)
print("Tool calls:", response.tool_calls)

In [ ]:
from langchain_openai import ChatOpenAI

llm_stateful = ChatOpenAI(
    model="gpt-4o-mini",
    use_responses_api=True,
)

respA = llm_stateful.invoke("Hi, I'm Bob. Please remember my name.")
print("Response A:", respA.content)
print("A's ID:", respA.response_metadata["id"])

respB = llm_stateful.invoke(
    "What is my name?",
    previous_response_id=respA.response_metadata["id"]
)
print("Response B:", respB.content)
